In [1]:
import sys
from pathlib import Path
srcdir = Path("../..").resolve()
print(f"Adding {srcdir} to sys.path, this is necessary to import from src")
sys.path.insert(0, str(srcdir))
print(sys.path)

import torch

Adding /Users/rgrouls/code/ML22 to sys.path, this is necessary to import from src
['/Users/rgrouls/code/ML22', '/Users/rgrouls/code/ML22/notebooks/4_tuning_networks', '/Users/rgrouls/.pyenv/versions/3.9.16/lib/python39.zip', '/Users/rgrouls/.pyenv/versions/3.9.16/lib/python3.9', '/Users/rgrouls/.pyenv/versions/3.9.16/lib/python3.9/lib-dynload', '', '/Users/rgrouls/Library/Caches/pypoetry/virtualenvs/deep-learning-HUU8cknU-py3.9/lib/python3.9/site-packages']


Let's revisit the flowers dataset from the first lesson

In [3]:
from mads_datasets import DatasetFactoryProvider, DatasetType

flowersfactory = DatasetFactoryProvider.create_factory(DatasetType.FLOWERS)
streamers = flowersfactory.create_datastreamer(batchsize=32)

2023-06-06 11:38:31.123 | INFO     | mads_datasets.datasetfactory:download_data:94 - Dataset already exists at /Users/rgrouls/.cache/mads_datasets/flowers
2023-06-06 11:38:31.648 | INFO     | mads_datasets.datasetfactory:download_data:104 - Digest of downloaded /Users/rgrouls/.cache/mads_datasets/flowers/flowers.tgz matches expected digest


In [4]:
streamers

{'train': BasetDatastreamer: ImgDataset (imgsize (224, 224), #classes 5) (streamerlen 91),
 'valid': BasetDatastreamer: ImgDataset (imgsize (224, 224), #classes 5) (streamerlen 22)}

In [11]:
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

The FlowersDatasetFactory adds a transform function that will (at random) flip, scale, affine an image. For the model, this is a new image, so it needs to focus on the general patterns instead of pixels.

In [12]:
X, y = next(trainstreamer)
X.shape, y.shape


TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [8]:
import matplotlib.pyplot as plt
img = X.permute(0, 2, 3, 1)
fig, axs = plt.subplots(3, 3, figsize=(10,10))
axs = axs.ravel()
for i in range(9):
    axs[i].imshow(img[i])

NameError: name 'X' is not defined

Let's check the ranges, mean and std of a batch

In [9]:
X.max(), X.min(), X.mean(), X.std()


(tensor(1.), tensor(0.), tensor(0.3832), tensor(0.3007))

Instead of building our own resnet, we will just download a pretrained version. This saves us many hours of training.

In [11]:
import torchvision
from torchvision.models import resnet18, ResNet18_Weights
resnet = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)


In [12]:
ResNet18_Weights.DEFAULT

ResNet18_Weights.IMAGENET1K_V1

In [13]:
yhat = resnet(X)
yhat.shape


torch.Size([32, 1000])

However, the resnet is trained for 1000 classes. We have just 5...

We will swap the last layer and retrain the model.

First, we freeze all pretrained layers:

In [14]:
for name, param in resnet.named_parameters():
    param.requires_grad = False


If you study the resnet implementation on [github](https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py#L206) you can see that the last layer is named `.fc`, like this:

```
 self.fc = nn.Linear(512 * block.expansion, num_classes)
 ```

This is a Linear layer, mapping from 512 * block.expansion to num_classes.


so we will swap that for our own. To do so we need to figure out how many features go into the .fc layer.
We can retrieve the incoming amount of features for the current `.fc` with `.in_features`

In [15]:
print(type(resnet.fc))
in_features = resnet.fc.in_features
in_features

<class 'torch.nn.modules.linear.Linear'>


512

Let's swap that layer with a small, two layer, neural network

In [16]:
import torch.nn as nn

resnet.fc = nn.Sequential(
    nn.Linear(in_features, 5)
    # nn.Linear(in_features, 128), nn.ReLU(), nn.Dropout(0.1), nn.Linear(128, 5)
)


In [17]:
yhat = resnet(X)
yhat.shape


torch.Size([32, 5])

So, we have a fully trained resnet, but we added two layers at the end that transforms everything into 5 classes.
These layers are random, so we need to train them for some epochs

In [18]:
from src.models import metrics
accuracy = metrics.Accuracy()

This will take some time to train (about 4 min per epoch), you could scale down to amount of trainsteps to speed things up.

You will start with a fairly high learning rate (0.01), and if the learning stops, after patience epochs the learning rate gets halved.

In [20]:
len(train), len(valid)

(91, 22)

In [ ]:
from src.settings import TrainerSettings, ReportTypes

settings = TrainerSettings(
    epochs=10,
    metrics=[accuracy],
    logdir="modellog",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD],
)

trainer = train_model.Trainer(
    model=resnet, 
    settings=settings, 
    loss_fn=nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam, 
    traindataloader=trainstreamer, 
    validdataloader=validstreamer, 
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
    )